In [1]:
import pandas as pd
import numpy as np
import scipy.optimize as opt
import time

In [2]:
# Reading the data on python
ps3_data = pd.read_stata('PS3_data.dta')
ps3_data.head(n=5)

,id68,year,intid,relhh,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,...,redpregovinc,hsex,wsex,age,wage,hpersno,wpersno,hyrsed,wyrsed,pce
0,1,1967,1,Head,1200.0,2000.0,NaN,NaN,0,NaN,...,5614.0,1.0,2.0,52.0,46.0,1.0,2.0,8.0,8.0,0.0
1,2,1967,2,Head,0.0,0.0,NaN,NaN,0,NaN,...,0.0,1.0,2.0,56.0,57.0,1.0,2.0,3.0,3.0,0.0
2,3,1967,3,Head,0.0,0.0,NaN,NaN,0,NaN,...,0.0,1.0,2.0,77.0,64.0,1.0,2.0,NaN,3.0,0.0
3,4,1967,4,Head,1560.0,0.0,NaN,NaN,6,1.0,...,3280.0,1.0,2.0,45.0,44.0,1.0,2.0,8.0,5.0,0.0
4,5,1967,5,Head,2500.0,2000.0,NaN,NaN,3,1.0,...,7900.0,1.0,2.0,24.0,22.0,1.0,2.0,10.0,9.0,0.0


# Data Cleaning and creation of necessary variables

In [3]:
# Deleting cell with hannhrs=0
ps3_data=ps3_data[ps3_data.hannhrs !=0]

In [4]:
# Creating a new variable for wage per hour = wph

ps3_data['wph']=ps3_data.hlabinc / ps3_data.hannhrs

# drop if missing value for the wph
ps3_data = ps3_data[ps3_data.wph.isnull() != True ]

# Natural logarithm conversion of a variable

ps3_data['lnwage']=np.log(ps3_data.wph)

In [5]:
# Creating a variable for constant

ps3_data['const']=1

In [6]:
# Creating dummy variables for all cateries given for a categorical variable

race = pd.get_dummies(ps3_data['hrace'])

In [7]:
ps3_data = pd.concat([ps3_data, race], axis=1)
ps3_data

,id68,year,intid,relhh,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,...,wpersno,hyrsed,wyrsed,pce,wph,lnwage,const,1.0,2.0,3.0
11161,402,1971,1,Head,1523.0,0.0,62928.707031,NaN,0,1.0,...,2.0,12.0,12.0,0.247121,41.318916,3.721320,1,1,0,0
11162,446,1971,2,Head,520.0,0.0,1618.640747,NaN,0,NaN,...,NaN,17.0,NaN,0.247121,3.112771,1.135513,1,1,0,0
11164,461,1971,4,Head,2010.0,0.0,22660.970703,NaN,0,1.0,...,2.0,5.0,5.0,0.247121,11.274115,2.422509,1,1,0,0
11165,462,1971,5,Head,1960.0,0.0,12949.125977,NaN,0,NaN,...,2.0,5.0,8.0,0.247121,6.606697,1.888084,1,1,0,0
11166,1126,1971,8,Head,2860.0,0.0,29337.865234,NaN,1,NaN,...,2.0,16.0,12.0,0.247121,10.257995,2.328057,1,1,0,0
11167,1585,1971,10,Head,840.0,833.0,8627.355469,6628.333984,0,1.0,...,2.0,12.0,12.0,0.247121,10.270661,2.329291,1,1,0,0
11170,97,1971,15,Head,1960.0,0.0,19678.625000,NaN,0,1.0,...,2.0,6.0,8.0,0.247121,10.040114,2.306588,1,1,0,0
11171,237,1971,16,Head,604.0,0.0,2124.466064,NaN,0,NaN,...,NaN,8.0,NaN,0.247121,3.517328,1.257702,1,1,0,0
11172,669,1971,17,Head,683.0,0.0,3439.611816,NaN,0,NaN,...,NaN,12.0,NaN,0.247121,5.036035,1.616619,1,1,0,0
11173,284,1971,20,Head,2400.0,0.0,76885.437500,NaN,2,1.0,...,2.0,16.0,12.0,0.247121,32.035599,3.466848,1,1,0,0


In [8]:
# At this point category 8.0 got dropped and there is no Hispanic people in the sample. So, I don't have the variable 'Hispanic' in my model.
# Renaming the existing variables in the dataframe

ps3_data.rename (columns = {'hyrsed':'educ',1.0:'White',2.0:'Black',3.0:'Native_American',},inplace=True)

In [9]:
ps3_data.rename(columns={'Native_American':'Other_Race'}, inplace=True)

In [10]:
ps3_data.describe()

,id68,year,intid,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,hrace,...,wpersno,educ,wyrsed,pce,wph,lnwage,const,White,Black,Other_Race
count,90041.000000,90041.000000,90041.000000,90041.000000,90041.000000,9.004100e+04,45673.000000,90041.000000,74042.000000,89923.000000,...,63091.000000,89537.000000,62617.000000,90041.000000,90041.000000,90041.000000,90041.0,90041.000000,90041.000000,90041.000000
mean,1510.727402,1986.337879,3518.940594,2072.334961,767.537964,4.217924e+04,21973.662109,0.951544,1.097256,1.124495,...,65.339951,13.230185,13.031749,0.610144,20.659475,2.751247,1.0,0.896736,0.079575,0.022379
std,835.012733,8.821313,2313.991695,750.543823,915.022034,4.672406e+04,20506.021484,1.168194,0.355753,0.392191,...,80.976387,2.525720,2.228436,0.209193,24.797258,0.768539,0.0,0.304305,0.270635,0.147913
min,1.000000,1971.000000,1.000000,1.000000,0.000000,6.353981e-01,1.192780,0.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,0.247121,0.000204,-8.499092,1.0,0.000000,0.000000,0.000000
25%,783.000000,1979.000000,1677.000000,1805.000000,0.000000,1.991069e+04,8078.967285,0.000000,1.000000,1.000000,...,2.000000,12.000000,12.000000,0.421747,10.415504,2.343295,1.0,1.000000,0.000000,0.000000
50%,1541.000000,1986.000000,3326.000000,2065.000000,81.699997,3.465334e+04,18154.320312,0.000000,1.000000,1.000000,...,4.000000,12.000000,12.000000,0.614522,16.558668,2.806910,1.0,1.000000,0.000000,0.000000
75%,2240.000000,1994.000000,5062.000000,2456.000000,1710.000000,5.273935e+04,30256.060547,2.000000,1.000000,1.000000,...,170.000000,16.000000,15.000000,0.803488,24.736206,3.208268,1.0,1.000000,0.000000,0.000000
max,2930.000000,2002.000000,16968.000000,7800.000000,5840.000000,3.771521e+06,417271.468750,11.000000,8.000000,3.000000,...,231.000000,17.000000,17.000000,0.928007,1865.037964,7.531037,1.0,1.000000,1.000000,1.000000


In [11]:
# Creating a subset of the dataset incorporating necessary variables

ps3_data1=ps3_data[['year','hsex','age','educ','wph','White','Black','Other_Race','lnwage','const']]
ps3_data1

,year,hsex,age,educ,wph,White,Black,Other_Race,lnwage,const
11161,1971,1.0,51.0,12.0,41.318916,1,0,0,3.721320,1
11162,1971,2.0,62.0,17.0,3.112771,1,0,0,1.135513,1
11164,1971,1.0,55.0,5.0,11.274115,1,0,0,2.422509,1
11165,1971,1.0,59.0,5.0,6.606697,1,0,0,1.888084,1
11166,1971,1.0,25.0,16.0,10.257995,1,0,0,2.328057,1
11167,1971,1.0,67.0,12.0,10.270661,1,0,0,2.329291,1
11170,1971,1.0,63.0,6.0,10.040114,1,0,0,2.306588,1
11171,1971,2.0,65.0,8.0,3.517328,1,0,0,1.257702,1
11172,1971,2.0,66.0,12.0,5.036035,1,0,0,1.616619,1
11173,1971,1.0,39.0,16.0,32.035599,1,0,0,3.466848,1


In [12]:
# Sorting the data according to requirement

ps_male = ps3_data1[ps3_data1.hsex==1]
ps_male_age=ps_male[(ps_male.age >= 25) & (ps_male.age <= 60)]
ps_male_age_wph=ps_male_age[ps_male_age.wph > 7]

In [13]:
ps_male_age_wph.describe()

,year,hsex,age,educ,wph,White,Black,Other_Race,lnwage,const
count,57477.000000,57477.0,57477.000000,57097.000000,57477.000000,57477.000000,57477.000000,57477.000000,57477.000000,57477.0
mean,1986.635245,1.0,39.224247,13.529993,24.306034,0.918907,0.056388,0.023035,3.010414,1.0
std,8.744894,0.0,9.579065,2.449510,25.154028,0.272981,0.230671,0.150017,0.543891,0.0
min,1971.000000,1.0,25.000000,1.000000,7.000252,0.000000,0.000000,0.000000,1.945946,1.0
25%,1979.000000,1.0,31.000000,12.000000,13.947624,1.000000,0.000000,0.000000,2.635309,1.0
50%,1987.000000,1.0,38.000000,13.000000,19.905161,1.000000,0.000000,0.000000,2.990979,1.0
75%,1994.000000,1.0,47.000000,16.000000,27.787226,1.000000,0.000000,0.000000,3.324576,1.0
max,2002.000000,1.0,60.000000,17.000000,1717.330322,1.000000,1.000000,1.000000,7.448526,1.0


In [14]:
ps_male_age_wph.columns

Index(['year', 'hsex', 'age', 'educ', 'wph', 'White', 'Black', 'Other_Race',
       'lnwage', 'const'],
      dtype='object')

In [15]:
ps_male_age_wph

,year,hsex,age,educ,wph,White,Black,Other_Race,lnwage,const
11161,1971,1.0,51.0,12.0,41.318916,1,0,0,3.721320,1
11164,1971,1.0,55.0,5.0,11.274115,1,0,0,2.422509,1
11166,1971,1.0,25.0,16.0,10.257995,1,0,0,2.328057,1
11173,1971,1.0,39.0,16.0,32.035599,1,0,0,3.466848,1
11175,1971,1.0,36.0,12.0,10.103716,1,0,0,2.312903,1
11192,1971,1.0,31.0,17.0,18.994255,1,0,0,2.944137,1
11196,1971,1.0,42.0,11.0,9.600762,0,1,0,2.261842,1
11198,1971,1.0,36.0,11.0,16.548191,1,0,0,2.806277,1
11199,1971,1.0,44.0,9.0,16.519394,1,0,0,2.804535,1
11202,1971,1.0,55.0,7.0,20.233009,1,0,0,3.007315,1


# Creating new data sets by different years

In [16]:
ps_1971=ps_male_age_wph[ps_male_age_wph.year==1971]

In [17]:
ps_1980=ps_male_age_wph[ps_male_age_wph.year==1980]

In [18]:
ps_1990=ps_male_age_wph[ps_male_age_wph.year==1990]

In [19]:
ps_2000=ps_male_age_wph[ps_male_age_wph.year==2000]

# OLS output for different years

In [20]:
import statsmodels.api as sm

C:\Users\mmic\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Year 1971

In [21]:
reg1=sm.OLS(endog=ps_1971['lnwage'], exog=ps_1971[['const','educ','age','Black','Other_Race']],missing='drop')

In [22]:
results1 = reg1.fit()

In [23]:
type (results1)
print(results1.summary())

                            OLS Regression Results                            
Dep. Variable:                 lnwage   R-squared:                       0.244
Model:                            OLS   Adj. R-squared:                  0.241
Method:                 Least Squares   F-statistic:                     110.7
Date:                Tue, 26 Sep 2017   Prob (F-statistic):           7.42e-82
Time:                        11:16:45   Log-Likelihood:                -728.06
No. Observations:                1380   AIC:                             1466.
Df Residuals:                    1375   BIC:                             1492.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5510      0.073     21.382      0.0

Year 1980

In [24]:
reg2=sm.OLS(endog=ps_1980['lnwage'], exog=ps_1980[['const','educ','age','Black','Other_Race']],missing='drop')

In [25]:
results2 = reg2.fit()
type (results2)
print(results2.summary())

                            OLS Regression Results                            
Dep. Variable:                 lnwage   R-squared:                       0.169
Model:                            OLS   Adj. R-squared:                  0.167
Method:                 Least Squares   F-statistic:                     94.08
Date:                Tue, 26 Sep 2017   Prob (F-statistic):           6.41e-73
Time:                        11:16:45   Log-Likelihood:                -1148.4
No. Observations:                1856   AIC:                             2307.
Df Residuals:                    1851   BIC:                             2334.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.6131      0.075     21.590      0.0

Year 1990

In [26]:
reg3=sm.OLS(endog=ps_1990['lnwage'], exog=ps_1990[['const','educ','age','Black','Other_Race']],missing='drop')

In [27]:
results3 = reg3.fit()
type (results3)
print(results3.summary())

                            OLS Regression Results                            
Dep. Variable:                 lnwage   R-squared:                       0.217
Model:                            OLS   Adj. R-squared:                  0.216
Method:                 Least Squares   F-statistic:                     139.3
Date:                Tue, 26 Sep 2017   Prob (F-statistic):          3.67e-105
Time:                        11:16:45   Log-Likelihood:                -1393.9
No. Observations:                2013   AIC:                             2798.
Df Residuals:                    2008   BIC:                             2826.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1186      0.084     13.312      0.0

Year 2000

In [28]:
reg4=sm.OLS(endog=ps_2000['lnwage'], exog=ps_2000[['const','educ','age','Black','Other_Race']],missing='drop')

In [29]:
results4 = reg4.fit()
type (results4)
print(results4.summary())

                            OLS Regression Results                            
Dep. Variable:                 lnwage   R-squared:                       0.207
Model:                            OLS   Adj. R-squared:                  0.205
Method:                 Least Squares   F-statistic:                     168.6
Date:                Tue, 26 Sep 2017   Prob (F-statistic):          1.75e-128
Time:                        11:16:45   Log-Likelihood:                -2081.4
No. Observations:                2595   AIC:                             4173.
Df Residuals:                    2590   BIC:                             4202.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1611      0.080     14.436      0.0

Interpretation of education coefficient for different years by OLS

Given one year increase in the years of education of the household head, we expect the wage to rise by 6.69 percentage, 6.76 percentage, 9.76 percentage, and 10.92 percentage in years 1971, 1980,1990, and 2000 respectivley.

# Loglikelihood Optimization

# Year 1971

In [30]:
def loglik71(b):
    '''
    This function returns the loglikelihood value for given betas and sigma (b) for year 1971.
    '''
    y=ps_1971['lnwage']
    yhat=b[0]+b[1]*ps_1971['educ']+b[2]*ps_1971['age']+b[3]*ps_1971['Black']+b[4]*ps_1971['Other_Race']
    e=y-yhat
    LL = -(len(e)/2)*np.log(2*np.pi)-len(e)*np.log(b[5])-(1/(2*b[5]**2))*((e*e).sum())
    return -LL

In [31]:
# Here b[5] = sigma
b0=[2,0.1,0.02,0.06,0.1,0.1]

In [32]:
# Applying bounds=bnds for the parameters where bound for sigma is (0,np.inf)

bnds = ((-np.inf, np.inf),(-np.inf, np.inf), (-np.inf, np.inf),(-np.inf, np.inf), (-np.inf, np.inf), (0, np.inf))

In [33]:
results_SLSQP1 = opt.minimize(loglik71,b0,bounds=bnds, method='SLSQP')
print(results_SLSQP1)

     fun: 15995.242818855651
     jac: array([ 0.        ,  0.        ,  0.        , -0.00390625, -0.00146484,
        0.00146484])
 message: 'Optimization terminated successfully.'
    nfev: 170
     nit: 19
    njev: 19
  status: 0
 success: True
       x: array([  2.00000000e+00,  -1.71199106e+02,  -4.93293359e+02,
         8.75998335e+01,   3.92915666e+01,   2.26046431e+04])


In [34]:
results_BFGS1 = opt.minimize(loglik71,b0,bounds=bnds, method='L-BFGS-B',tol=1e-15)
print(results_BFGS1)

      fun: nan
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([   0.        , -164.26492948,  748.25716183,  127.68060742,
         18.06280352,   34.44326921])
  message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 469
      nit: 19
   status: 2
  success: False
        x: array([ 2.        ,  0.05068645,  0.00814371,  0.05580784,  0.09932973,
        0.42053305])


C:\Users\mmic\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  
C:\Users\mmic\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in double_scalars
  
C:\Users\mmic\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


In [35]:
results_NM1 = opt.minimize(loglik71,b0, method='Nelder-Mead',tol=1e-15)
print(results_NM1)

 final_simplex: (array([[ 1.55068243,  0.06689189,  0.01439381, -0.16373869,  0.03053483,
         0.40729411],
       [ 1.55068241,  0.06689189,  0.01439381, -0.16373862,  0.03053477,
         0.40729413],
       [ 1.55068242,  0.06689189,  0.01439381, -0.16373863,  0.03053475,
         0.40729413],
       [ 1.5506824 ,  0.06689189,  0.01439381, -0.16373866,  0.03053473,
         0.40729412],
       [ 1.55068241,  0.06689189,  0.01439381, -0.16373859,  0.03053475,
         0.40729413],
       [ 1.55068239,  0.06689189,  0.01439381, -0.16373856,  0.03053468,
         0.40729415],
       [ 1.55068241,  0.06689189,  0.01439381, -0.1637386 ,  0.03053464,
         0.40729414]]), array([ 728.52180621,  728.52180621,  728.52180621,  728.52180621,
        728.52180621,  728.52180621,  728.52180621]))
           fun: 728.52180620690001
       message: 'Maximum number of function evaluations has been exceeded.'
          nfev: 1201
           nit: 745
        status: 1
       success: False
   

# Year 1980

In [36]:
def loglik80(b):
    '''
    This function returns the loglikelihood value for given betas and sigma (b) for year 1980.
    '''
    y=ps_1980['lnwage']
    yhat=b[0]+b[1]*ps_1980['educ']+b[2]*ps_1980['age']+b[3]*ps_1980['Black']+b[4]*ps_1980['Other_Race']
    e=y-yhat
    LL = -(len(e)/2)*np.log(2*np.pi)-len(e)*np.log(b[5])-(1/(2*b[5]**2))*((e*e).sum())
    return -LL

In [37]:
results_SLSQP2 = opt.minimize(loglik80,b0,bounds=bnds, method='SLSQP')
print(results_SLSQP2)

     fun: 21668.607453399633
     jac: array([ 0.        ,  0.        ,  0.        , -0.00366211, -0.0012207 ,
        0.00341797])
 message: 'Optimization terminated successfully.'
    nfev: 161
     nit: 18
    njev: 18
  status: 0
 success: True
       x: array([  2.00000000e+00,  -2.32615419e+02,  -6.37483419e+02,
         6.65387443e+01,   2.89522934e+01,   2.90296660e+04])


In [38]:
results_BFGS2 = opt.minimize(loglik80,b0,bounds=bnds, method='L-BFGS-B',tol=1e-15)
print(results_BFGS2)

      fun: nan
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([    0.        ,  3167.13947086, -1743.15125605,    -4.74365152,
          11.84303073,    36.16989943])
  message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 721
      nit: 30
   status: 2
  success: False
        x: array([ 2.        ,  0.04282668,  0.01087241, -0.13584097,  0.07702936,
        0.45556698])


C:\Users\mmic\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  
C:\Users\mmic\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in double_scalars
  
C:\Users\mmic\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


In [39]:
results_NM2 = opt.minimize(loglik80,b0, method='Nelder-Mead',tol=1e-15)
print(results_NM2)

 final_simplex: (array([[ 1.61027294,  0.06770588,  0.01272869, -0.09456881, -0.02163746,
         0.44917999],
       [ 1.61027294,  0.06770588,  0.01272869, -0.09456881, -0.02163746,
         0.44917999],
       [ 1.61027294,  0.06770588,  0.01272869, -0.09456881, -0.02163746,
         0.44917999],
       [ 1.61027294,  0.06770588,  0.01272869, -0.09456881, -0.02163746,
         0.44917999],
       [ 1.61027294,  0.06770588,  0.01272869, -0.09456881, -0.02163746,
         0.44917999],
       [ 1.61027294,  0.06770588,  0.01272869, -0.09456881, -0.02163746,
         0.44917999],
       [ 1.61027294,  0.06770588,  0.01272869, -0.09456881, -0.02163746,
         0.44917999]]), array([ 1148.65611911,  1148.65611911,  1148.65611911,  1148.65611911,
        1148.65611911,  1148.65611911,  1148.65611911]))
           fun: 1148.6561191070671
       message: 'Maximum number of function evaluations has been exceeded.'
          nfev: 1201
           nit: 682
        status: 1
       success: Fa

# Year 1990

In [40]:
def loglik90(b):
    '''
    This function returns the loglikelihood value for given betas and sigma (b) for year 1990.
    '''
    y=ps_1990['lnwage']
    yhat=b[0]+b[1]*ps_1990['educ']+b[2]*ps_1990['age']+b[3]*ps_1990['Black']+b[4]*ps_1990['Other_Race']
    e=y-yhat
    LL = -(len(e)/2)*np.log(2*np.pi)-len(e)*np.log(b[5])-(1/(2*b[5]**2))*((e*e).sum())
    return -LL

In [41]:
results_SLSQP3 = opt.minimize(loglik90,b0,bounds=bnds, method='SLSQP')
print(results_SLSQP3)

     fun: 23788.53191444083
     jac: array([ 0.        ,  0.        ,  0.        , -0.00317383, -0.00073242,
        0.00024414])
 message: 'Optimization terminated successfully.'
    nfev: 165
     nit: 18
    njev: 18
  status: 0
 success: True
       x: array([  2.00000000e+00,  -2.77845818e+02,  -7.12583320e+02,
         5.51611627e+01,   1.67380986e+01,   3.22114618e+04])


In [42]:
results_BFGS3 = opt.minimize(loglik90,b0,bounds=bnds, method='L-BFGS-B',tol=1e-15)
print(results_BFGS3)

      fun: nan
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([  0.00000000e+00,   1.21096846e+04,  -1.47015994e+03,
         1.08554855e+02,   1.39077201e+01,   1.14852355e+01])
  message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 672
      nit: 26
   status: 2
  success: False
        x: array([ 2.        ,  0.02209948,  0.01704091,  0.0537477 ,  0.09909537,
        0.51755757])


C:\Users\mmic\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  
C:\Users\mmic\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in double_scalars
  
C:\Users\mmic\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


In [43]:
results_NM3 = opt.minimize(loglik90,b0, method='Nelder-Mead',tol=1e-15)
print(results_NM3)

 final_simplex: (array([[ 1.11840719,  0.09756539,  0.01346719, -0.17203877, -0.05980211,
         0.48323804],
       [ 1.11840719,  0.09756539,  0.01346719, -0.17203879, -0.05980213,
         0.48323804],
       [ 1.11840719,  0.09756539,  0.01346719, -0.17203881, -0.05980215,
         0.48323804],
       [ 1.11840719,  0.09756539,  0.01346719, -0.17203881, -0.05980216,
         0.48323804],
       [ 1.11840719,  0.09756539,  0.01346719, -0.17203884, -0.05980218,
         0.48323804],
       [ 1.11840719,  0.09756539,  0.01346719, -0.17203885, -0.05980219,
         0.48323804],
       [ 1.11840719,  0.09756539,  0.01346719, -0.17203886, -0.05980221,
         0.48323804]]), array([ 1394.45831523,  1394.45831523,  1394.45831523,  1394.45831523,
        1394.45831523,  1394.45831523,  1394.45831523]))
           fun: 1394.4583152308287
       message: 'Maximum number of function evaluations has been exceeded.'
          nfev: 1200
           nit: 706
        status: 1
       success: Fa

# Year 2000

In [44]:
def loglik00(b):
    '''
    This function returns the loglikelihood value for given betas and sigma (b) for year 2000.
    '''
    y=ps_2000['lnwage']
    yhat=b[0]+b[1]*ps_2000['educ']+b[2]*ps_2000['age']+b[3]*ps_2000['Black']+b[4]*ps_2000['Other_Race']
    e=y-yhat
    LL = -(len(e)/2)*np.log(2*np.pi)-len(e)*np.log(b[5])-(1/(2*b[5]**2))*((e*e).sum())
    return -LL

In [45]:
results_SLSQP4 = opt.minimize(loglik00,b0,bounds=bnds, method='SLSQP')
print(results_SLSQP4)

     fun: 32343.932554172494
     jac: array([ 0.        ,  0.        ,  0.        , -0.00292969, -0.00195312,
        0.00024414])
 message: 'Optimization terminated successfully.'
    nfev: 168
     nit: 19
    njev: 19
  status: 0
 success: True
       x: array([  2.00000000e+00,  -3.46494445e+02,  -9.28356145e+02,
         7.25415963e+01,   4.57813106e+01,   4.34467573e+04])


In [46]:
results_BFGS4 = opt.minimize(loglik00,b0,bounds=bnds, method='L-BFGS-B',tol=1e-15)
print(results_BFGS4)

      fun: 2162.505136314634
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([  0.00000000e+00,   7.92387436e+02,   3.02240715e+03,
        -6.54667929e+00,   2.11662154e+01,  -6.53562893e-01])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 973
      nit: 40
   status: 0
  success: True
        x: array([ 2.        ,  0.0723178 ,  0.00359789, -0.31838234, -0.02218935,
        0.54319361])


In [47]:
results_NM4 = opt.minimize(loglik00,b0, method='Nelder-Mead',tol=1e-15)
print(results_NM4)

 final_simplex: (array([[ 1.16381484,  0.10915288,  0.01094126, -0.26323987, -0.03960205,
         0.5313921 ],
       [ 1.16381484,  0.10915288,  0.01094126, -0.26323987, -0.03960205,
         0.5313921 ],
       [ 1.16381484,  0.10915288,  0.01094126, -0.26323987, -0.03960205,
         0.5313921 ],
       [ 1.16381484,  0.10915288,  0.01094126, -0.26323987, -0.03960205,
         0.5313921 ],
       [ 1.16381484,  0.10915288,  0.01094126, -0.26323987, -0.03960205,
         0.5313921 ],
       [ 1.16381484,  0.10915288,  0.01094126, -0.26323987, -0.03960205,
         0.5313921 ],
       [ 1.16381484,  0.10915288,  0.01094126, -0.26323987, -0.03960205,
         0.5313921 ]]), array([ 2104.77503873,  2104.77503873,  2104.77503873,  2104.77503873,
        2104.77503873,  2104.77503873,  2104.77503873]))
           fun: 2104.7750387273677
       message: 'Optimization terminated successfully.'
          nfev: 1157
           nit: 608
        status: 0
       success: True
             x: a

# Interpretation of education coefficient

Using Nelder-Mead Method

Given one year increase in the years of education of the household head, we expect the wage to rise by 6.69 percentage, 6.77 percentage, 9.76 percentage, and 10.92 percentage in years 1971, 1980,1990, and 2000 respectivley.

* Results using other two methods (SLSQP and L-BFGS-B) are not consistent with the OLS result. So, I didn't interpret those results.